TO-DO

1) Посмтроить гистограмму истинных Y, посмотреть, номральное ли у них распределение

2) Добавить Dropout




УСТАНОВКА ВСЕХ НЕОБХОДИМЫХ ЗАВИСИМОСТЕЙ

In [ ]:
!pip install -U keras-tuner

import pandas as pd
import numpy as np
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from kerastuner.engine import hyperparameters
from keras.layers import Dropout
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import time
import math
import matplotlib.pyplot as plt
from keras import utils


ЧТЕНИЕ И ВЫВОД ИСХОДНЫХ ДАННЫХ


In [ ]:
TRAIN_FILE_PATH = '/content/drive/My Drive/Интеллектуальный анализ данных ЛР№1/train.csv'

# загружаем данные
x = pd.read_csv(TRAIN_FILE_PATH) 

x

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,s,as,c,d,aa,d,q,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,o,t,d,d,aa,h,h,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,v,r,a,d,aa,g,e,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,r,e,f,d,aa,l,u,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


ПОЛУЧЕНИЕ ЗНАЧЕНИЙ Y

In [ ]:
# функция извлекает правильные значение из тренировочных данных
def get_truth(array):
  data = []
  for col_name, col_info in array.items():
    # находится столбец y
    if col_name == "y":
      # все данные из него пререписываются в массив
      for line_number, line_info in col_info.items():
        temp_array = []
        temp_array.append(line_info)
        data.append(temp_array)
  # массив трансформируется в numpy и меняется его размерность
  return np.array(data, dtype=np.float64).reshape(array.shape[0], 1)


y = get_truth(x)
y

array([[130.81],
       [ 88.53],
       [ 76.26],
       ...,
       [109.22],
       [ 87.48],
       [110.85]])

УДАЛЕНИЕ ЛИШНИХ СТОЛБЦОВ

In [ ]:
# столбцы 'y' и 'ID' не нужны при работе больше
x.drop('y', axis='columns', inplace=True)
x.drop('ID', axis='columns', inplace=True)
x

ШИФРОВАНИЕ ВСЕХ БУКВ И ЦИФР С ПОМОЩЬЮ LABELENCODER


In [ ]:
# все буквы и цифры шифруются, чтобы остались только числа
df = pd.DataFrame(x)
x = df.apply(preprocessing.LabelEncoder().fit_transform)

x

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,32,23,17,0,3,24,9,14,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,32,21,19,4,3,28,11,14,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,20,24,34,2,3,27,9,23,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,20,21,34,5,3,27,11,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,20,23,34,5,3,12,3,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8,20,16,2,3,0,3,16,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4205,31,16,40,3,3,0,7,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,8,23,38,0,3,0,6,4,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4207,9,19,25,5,3,0,11,20,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


НОРМАЛИЗАЦИЯ ДАННЫХ

In [ ]:
# функция нормализуем данные для работы
def normalize_data(data):
  mean = data.mean(axis=0)
  std = data.std(axis=0)
  data -= mean
  data /= std
  return data

  
x_norm = normalize_data(x)
x_norm

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0.162993,1.393322,-0.028118,-1.678071,0.028935,1.291964,0.751698,0.339405,-0.116108,NaN,-0.284872,4.03065,-0.865143,-0.021801,-0.051183,-0.087517,11.247906,-0.332457,-0.408087,19.533195,-0.30857,-0.145263,-0.043633,-0.070803,-1.466267,-0.183402,-0.211955,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,-4.237357,-0.216803,-0.715734,-0.650246,-0.239287,-0.046286,1.981827,-0.783412,-0.468232,-0.034483,-0.863046,-0.181316,-0.287786,0.187514,-1.041363,-1.749861,-0.053465,-0.053465,-0.034483,-0.233688,-0.258659,-0.021801,-0.081825,-0.120241,-0.021801,-0.140062,-0.542412,-0.684086,-0.246417,1.475156,-0.145263,-0.097941,-0.090232,-0.087517,-0.04081,-0.021801,-0.037778
1,0.162993,1.158883,0.155369,0.620896,0.028935,1.776763,1.437340,0.339405,-0.116108,NaN,-0.284872,-0.24804,-0.865143,-0.021801,-0.051183,-0.087517,11.247906,-0.332457,-0.408087,-0.051183,-0.30857,-0.145263,-0.043633,-0.070803,0.681842,-0.183402,-0.211955,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,0.235940,-0.216803,-0.715734,-0.650246,-0.239287,-0.046286,-0.504465,-0.783412,-0.468232,-0.034483,-0.863046,-0.181316,-0.287786,0.187514,-1.041363,0.571338,-0.053465,-0.053465,-0.034483,-0.233688,-0.258659,-0.021801,-0.081825,-0.120241,-0.021801,-0.140062,-0.542412,1.461457,-0.246417,-0.677733,-0.145263,-0.097941,-0.090232,-0.087517,-0.04081,-0.021801,-0.037778
2,-0.710475,1.510542,1.531527,-0.528587,0.028935,1.655563,0.751698,1.618197,-0.116108,NaN,-0.284872,-0.24804,-0.865143,-0.021801,-0.051183,11.423670,-0.088884,-0.332457,-0.408087,-0.051183,-0.30857,-0.145263,-0.043633,-0.070803,0.681842,5.451199,4.716869,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,0.235940,-0.216803,1.396835,-0.650246,-0.239287,-0.046286,1.981827,-0.783412,-0.468232,-0.034483,1.158412,-0.181316,-0.287786,0.187514,-1.041363,0.571338,-0.053465,-0.053465,-0.034483,-0.233688,-0.258659,-0.021801,-0.081825,-0.120241,-0.021801,-0.140062,-0.542412,-0.684086,-0.246417,-0.677733,-0.145263,-0.097941,-0.090232,11.423670,-0.04081,-0.021801,-0.037778
3,-0.710475,1.158883,1.531527,1.195637,0.028935,1.655563,1.437340,-1.081476,-0.116108,NaN,-0.284872,-0.24804,-0.865143,-0.021801,-0.051183,-0.087517,-0.088884,-0.332457,-0.408087,-0.051183,-0.30857,-0.145263,-0.043633,-0.070803,0.681842,5.451199,4.716869,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,0.235940,-0.216803,1.396835,-0.650246,-0.239287,-0.046286,-0.504465,-0.783412,-0.468232,-0.034483,1.158412,-0.181316,-0.287786,0.187514,-1.041363,0.571338,-0.053465,-0.053465,-0.034483,-0.233688,-0.258659,-0.021801,-0.081825,-0.120241,45.858481,-0.140062,-0.542412,-0.684086,-0.246417,-0.677733,-0.145263,-0.097941,-0.090232,-0.087517,-0.04081,-0.021801,-0.037778
4,-0.710475,1.393322,1.531527,1.195637,0.028935,-0.162435,-1.305229,0.197316,-0.116108,NaN,-0.284872,-0.24804,-0.865143,-0.021801,-0.051183,-0.087517,-0.088884,-0.332457,-0.408087,-0.051183,-0.30857,-0.145263,-0.043633,-0.070803,0.681842,5.451199,4.716869,-0.067331,1.817387,-0.106254,-0.015414,-0.074116,1.817387,-0.067331,1.817387,-0.185468,-0.015414,-0.026704,-0.107392,-0.015414,...,-0.151122,-0.223329,NaN,0.235940,-0.216803,1.396835,-0.650246,-0.239287,-0.046286,-0.504465,-0.783412,-0.468232,-0.034483,1.158412,-0.181316,-0.287786,0.187

ПРОВЕРКА НА НАЛИЧИЕ NaN

In [ ]:
def check_nans(data):
  # массив названий столбцов, в которых одни NaN
  nan_cols = []
  for col_name, col_info in data.items():
    for line_number, line_info in col_info.items():
      if math.isnan(line_info):
        print(f"{col_name } is NaNs")
        nan_cols.append(col_name)
        break
  if nan_cols == []:
    print("[INFO] THERE IS NO NaNS IN DATA!")
  return nan_cols

nan_cols = check_nans(x_norm)

X11 is NaNs
X93 is NaNs
X107 is NaNs
X233 is NaNs
X235 is NaNs
X268 is NaNs
X289 is NaNs
X290 is NaNs
X293 is NaNs
X297 is NaNs
X330 is NaNs
X347 is NaNs


УДАЛЕНИЕ ВСЕХ СТОЛБЦОВ С NaN

In [ ]:
def delete_nans(data, nan_cols):
  for col in nan_cols:
    try:
      x.drop(str(col), axis='columns', inplace=True)
    except:
      continue

delete_nans(x_norm, nan_cols)

ПОВТОРНАЯ ПРОВЕРКА НА NaN

In [ ]:
check_nans(x_norm)

[INFO] THERE IS NO NaNS IN DATA!


[]

ДЕЛИМ ДАННЫЕ НА ТРЕНИРОВОЧНЫЕ И ТЕСТОВЫЕ В СООТНОШЕНИИ 70/30

In [ ]:
# делим в соотношении 70 на 30
(x_train, x_test) = train_test_split(x_norm, test_size=0.3)
(y_train, y_test) = train_test_split(y, test_size=0.3)

ПРЕОБРАЗОВАНИЕ ВСЕХ ДАННЫХ В NUMPY

In [ ]:

# функция принимает данные, прочитанные из csv файла, переводит их в numpy_array
def convert_csv_to_numpy(array):
  array_to_convert = []
  for col_name, col_info in array.items():
    # временный массив создается для каждого СТОЛБЦА
    temp_array = []
    for line_number, line_info in col_info.items():
        temp_array.append(line_info)
    # после прохождения всего столбца временный массив добавляется в основной
    if temp_array != []:
      array_to_convert.append(temp_array)
  # в самом конце основном массив конвертируется в numpy_array
  return np.array(array_to_convert, dtype=np.float64)


x_train = convert_csv_to_numpy(x_train) 
x_test = convert_csv_to_numpy(x_test)


In [ ]:
x_train

array([[-1.1472094 , -1.51115443,  1.18203883, ...,  1.10924982,
         0.9636718 , -0.78326436],
       [-0.83384897,  0.92444416, -1.18550759, ...,  1.39332233,
         0.22112691, -0.13053172],
       [-0.11986205,  0.7058325 , -0.11986205, ...,  0.79757634,
        -1.31253196, -0.11986205],
       ...,
       [-0.04081026, -0.04081026, -0.04081026, ..., -0.04081026,
        -0.04081026, -0.04081026],
       [-0.02180104, -0.02180104, -0.02180104, ..., -0.02180104,
        -0.02180104, -0.02180104],
       [-0.03777847, -0.03777847, -0.03777847, ..., -0.03777847,
        -0.03777847, -0.03777847]])

In [ ]:
x_test

array([[-1.51115443, -0.78326436, -1.51115443, ..., -1.29278741,
        -0.20095231,  0.8908828 ],
       [ 0.92444416, -1.18550759,  0.92444416, ..., -0.83384897,
         0.22112691,  1.0416637 ],
       [ 0.7058325 , -0.11986205,  1.07280786, ...,  1.99024625,
        -0.11986205, -0.11986205],
       ...,
       [-0.04081026, -0.04081026, -0.04081026, ..., -0.04081026,
        -0.04081026, -0.04081026],
       [-0.02180104, -0.02180104, -0.02180104, ..., -0.02180104,
        -0.02180104, -0.02180104],
       [-0.03777847, -0.03777847, -0.03777847, ..., -0.03777847,
        -0.03777847, -0.03777847]])

ИЗМЕНЕНИЕ ФОРМЫ ТЕНЗОРОВ

In [ ]:
# меняю формы обоих x, чтобы первое измерение совпадало с y
x_train = np.reshape(x_train, (x_train.shape[1], x_train.shape[0]))
x_test = np.reshape(x_test, (x_test.shape[1], x_test.shape[0]))

print(f"Y_TRAIN SHAPE IS {y_train.shape}")
print(f"Y_TEST SHAPE IS {y_test.shape}")
print(f"X_TRAIN SHAPE IS {x_train.shape}")
print(f"X_TEST SHAPE IS {x_test.shape}")
print("\n\n")


Y_TRAIN SHAPE IS (2946, 1)
Y_TEST SHAPE IS (1263, 1)
X_TRAIN SHAPE IS (2946, 364)
X_TEST SHAPE IS (1263, 364)





БЕРЕМ 30 ПРОЦЕНТОВ ТРЕНИРОВОЧНЫХ ДАННЫХ ДЛЯ ПРОВЕРКИ **ВО ВРЕМЯ ТРЕНИРОВКИ**

In [ ]:
(x_train_main, x_train_val) = train_test_split(x_train, test_size=0.3)
(y_train_main, y_train_val) = train_test_split(y_train, test_size=0.3)

СОЗДАНИЕ И ОБУЧЕНИЕ МОДЕЛИ ВРУЧНУЮ (ПОКА ЧТО ЛУЧШИЙ ВАРИАНТ)

In [ ]:
model = Sequential()
model.add(Dense(1024, activation="relu", input_shape=(x_train_main.shape[1],)))
model.add(Dense(512, activation="relu"))
model.add(Dense(1))
print("[INFO] NN has been created")
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
print("[INFO] NN has been compiled")
model.summary()
# начинаем обучение нейронной сети
history = model.fit(x_train_main, y_train_main, epochs=100, batch_size=100, validation_data=(x_train_val, y_train_val))
history = history.history
print("[INFO] Training has been finished")


In [ ]:
model = Sequential()
model.add(Dense(1024, activation="relu", input_shape=(x_train_main.shape[1],)))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1))
print("[INFO] NN has been created")
model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
print("[INFO] NN has been compiled")
model.summary()
# начинаем обучение нейронной сети
history = model.fit(x_train_main, y_train_main, epochs=100, batch_size=100, validation_data=(x_train_val, y_train_val))
history = history.history
print("[INFO] Training has been finished")


СОЗДАНИЕ ГРАФИКОВ

In [ ]:
# функция строит графики после тренировки
def draw_graph(history):
    loss_values = history["loss"]
    validation_loss_values = history["val_loss"]

    epochs = range(1, len(history['loss']) + 1)

    #               ГРАФИКИ ПОТЕРЬ
    # синими точками рисуется график потерь на этапе обучения
    plt.plot(epochs, loss_values, 'or', label='Training loss')
    # синей линией рисуется график потерь на этапе проверки
    plt.plot(epochs, validation_loss_values, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # очищаем рисунок
    plt.clf()

    #               ГРАФИКИ ОШИБКИ
    acc_values = history['mae']
    validation_acc_values = history['val_mae']
    plt.plot(epochs, acc_values, 'or', label='Training mae')
    plt.plot(epochs, validation_acc_values, 'b', label='Validation mae')
    plt.title('Training and validation mae')
    plt.xlabel('Epochs')
    plt.ylabel('mae')
    plt.legend()
    plt.show()



# рисуем все графики
draw_graph(history)

ВЫЧИСЛЕНИЕ ОШИБКИ ТРЕНИРОВКИ

In [ ]:
print("[INFO] RUNNING ON TEST DATA: \n")
# вычисляем ошибки и выводим их на экран
mse, mae = model.evaluate(x_test, y_test, verbose=1)
print(f"[INFO] Mean squared error is {mse}")
print(f"[INFO] Mean absolute error is {mae}")

СОЗДАНИЕ И ОБУЧЕНИЕ МОДЕЛИ ЧЕРЕЗ KERAS TUNER

In [ ]:
# функция создание нейронной сети
def build_NN(hp):
  model = Sequential()
  # варианты активационых функций
  activation_choice = hp.Choice("activation", values=["relu", "tanh"])
  # создаем один входной слой
  model.add(Dense(units=hp.Int("units_input", min_value=4,max_value=128, step=32), activation=activation_choice, input_dim=364))
  # опционально добавляются еще слои разной конфигурации
  for i in range(hp.Int("n_layers", 2, 3)):
    model.add(Dense(units=hp.Int("units_"+str(i), min_value=4, max_value=128, step=32), activation=activation_choice))
  # создаем один выходной слой
  model.add(Dense(1))
  # компиляция сети происходит с выборо одного из оптимизаторов
  model.compile(optimizer=hp.Choice("optimizer", values=["rmsprop"]), loss="mse", metrics=["mae"])
  model.summary()
  return model


def find_best_NN(x_train_main, y_train_main):
  # создаю тюнер, который сможет подобрать оптимальную архитектуру модели
  tuner = RandomSearch(build_NN, objective="loss", max_trials=20, executions_per_trial=1)
  print("\n\n\n")
  # начинается автоматический подбор гиперпараметров
  print('[INFO] start searching')
  tuner.search(x_train_main, y_train_main, batch_size=100, epochs=10, validation_data=(x_train_val, y_train_val))
  # выбираем лучшую модель
  print("\n\n\nRESULTS SUMMARY")
  tuner.results_summary()
  print("\n\n\n")
  # получаем лучшую модель
  print("\n\n\nHERE IS THE BEST MODEL\n\n\n")
  best_model = tuner.get_best_models(num_models=1)[0]

  return best_model
  

best_model = find_best_NN(x_train_main, y_train_main)


ТРЕНИРОВКА НАИЛУЧШЕЙ МОДЕЛИ

In [ ]:
best_history = best_model.fit(x_train_main, y_train_main, epochs=30, batch_size=100, validation_data=(x_train_val, y_train_val))
best_history = best_history.history
print("[INFO] Training has been finished")

ГРАФИКИ ТРЕНИРОВКИ ЛУЧШЕЙ МОДЕЛИ

In [ ]:
# рисуем все графики
draw_graph(best_history)

ПРОВЕРКА ЛУЧШЕЙ МОДЕЛИ НА ТЕСТОВЫХ ДАННЫХ

In [ ]:
mse, mae = best_model.evaluate(x_test, y_test, verbose=1)
print(f"[INFO] Mean squared error is {mse}")
print(f"[INFO] Mean absolute error is {mae}")

40/40 [==============================] - 0s 906us/step - loss: 433.1727 - mae: 15.8964
[INFO] Mean squared error is 433.1726989746094
[INFO] Mean absolute error is 15.896387100219727


ПРЕДСКАЗАНИЕ

In [ ]:
x = np.array(x_norm)
predicted_y = model.predict(x)
predicted_y

ПЕРЕВОДИМ В ВЕКТОРЫ ДЛЯ ПОДСЧЕТА КОРРЕЛЯЦИИ

In [ ]:
predicted_y = np.reshape(predicted_y, (4209))
y = np.reshape(y, (4209))


ПОДСЧЕТ КОРРЕЛЯЦИИ

если коэффициент = 1 или =-1 то зависимость сильная - так должно быть

если коэффициент примерно 0, то зависимость слабая - плохо

In [ ]:
kc = np.corrcoef(predicted_y, y)

print(kc)

[[ 1.         -0.03696418]
 [-0.03696418  1.        ]]
